In [2]:
from typing import Callable, Dict
import datetime

import pandas as pd
import numpy as np

from forexgym.utils import Query, available_timeframes, Timeframe, CurrencyPair

In [3]:
def default_processor(df: pd.DataFrame, time_column: str = "Date", *args, **kwargs) -> pd.DataFrame:
        """
            Drops the <time_column> column from an OHLC DataFrame. Default manipulation if no data_processor is provided in a query.

            Parameters:
                df (pd.DataFrame): The DataFrame from which to drop the <time_column> column.
                time_column (str): The name of the column to drop.
            
            Returns:
                pd.DataFrame: The OHLC with the <time_column> column removed.
        """
        
        return df.drop([time_column], axis=1)

In [101]:
def format_datasets(query: Query, timeframes: Dict[str, pd.DataFrame], time_column: str = "Date", *args, **kwargs) -> None:
        
    trading_timeframe_lable = query.trading_timeframe
    trading_timeframe = available_timeframes[trading_timeframe_lable]
    trading_column = query.trading_column
    
    trading_df = timeframes[trading_timeframe_lable]
    
    
    episode_data = pd.DataFrame()
    episode_data[time_column] = trading_df[time_column]
    episode_data["Trading_Price"] = trading_df[trading_column] # TODO: Include OHLC
    
    
    
    for query_params in query.queries:
        query_timeframe: Timeframe = query_params["timeframe"]
        window_size: int = query_params["window_size"]
        data_processor: Callable[[pd.DataFrame], pd.DataFrame] = query_params.get("data_processor", default_processor)
        
        # Selects the query dataframe
        query_df = timeframes[query_timeframe.lable]
        
        # Extracts features from the query dataframe
        processed_df = data_processor(query_df)
        
        # Creates a dataframe in which all rows contain all the features
        stacked = pd.concat([processed_df.add_suffix(f"_{shift}").shift(shift) for shift in range(window_size)], axis=1)
        
        # Smaller or equal than the trading TF
        if trading_timeframe >= query_timeframe:
            
            # Uses the existing dates in the trading dataframe to be included in the final datasets since all the other rows are irrelevant and will be discarded
            valid = query_df[time_column].isin(trading_df[time_column])
            
            # Selects the relevant rows (rows which share dates), adds a prefix and adds them to the final dataset
            filtered: pd.DataFrame = stacked[valid].reset_index(drop=True).add_prefix(f"{query_timeframe.lable}_")
            episode_data = pd.concat([episode_data, filtered], axis=1)
        
        # Greater TF than the trading TF 
        else:
            # Returns a pd.Series in which all dates of the trading dataframe have been rounded down to match that of the query (e.g. 12:00 -> 12:00, 12:15 -> 12:00, 12:30 -> 12:00, 12:45 -> 12:00)
            trading_dates = trading_df[time_column].copy().dt.floor(query_timeframe.value)
            
            # Includes the Date column back into the feature dataframe to be compared (the user probably has removed Date column from features)
            stacked[time_column] = query_df[time_column]
            
            # Given the trading dates (a series with Query Tf/Trading Tf n repeated rows 1H/15m = 4 repeated rows)
            # For each row (date) in trading dates the corresponding row (date) in the stacked dataframe is merged with the corresponding row (date) in the stacked dataframe such that the two rows share the same date
            # It therefore yields a dataframe consisting of merged rows with the same date so that for all divisiones in the trading TF it has data from the higher TFs
            # It is shifted n times to avoid lookahead bias
            # Ik its a bit confusing but it works
            
            
            filtered = pd.merge(trading_dates, stacked, on=time_column, how='outer').drop([time_column], axis=1).add_prefix(f"{query_timeframe.lable}_").shift(int(query_timeframe.value/trading_timeframe.value))
            
            # Filtered TF added horizontaly into the final dataset
            episode_data = pd.concat([episode_data, filtered], axis=1)
    
   
    episode_data = episode_data.dropna().reset_index(drop=True)
    
    
    
    return episode_data

In [95]:
def select_close(df: pd.DataFrame, *args, **kwargs) -> pd.DataFrame:
    
    return pd.DataFrame(df["Close"])

def select_date(df: pd.DataFrame, *args, **kwargs) -> pd.DataFrame:
    
    return df.drop(["Open", "High", "Low"], axis=1)

def article_processor(df: pd.DataFrame, *args, **kwargs) -> pd.DataFrame:
    df["x1"] = ((df["Close"] - df["Close"].shift(1) ) / df["Close"]).shift(1) 
    df["x2"] = ((df["High"] - df["High"].shift(1) ) / df["High"]).shift(1) 
    df["x3"] = ((df["Low"] - df["Low"].shift(1) ) / df["Low"]).shift(1) 
    df["x4"] = (df["High"] - df["Close"]) / df["Close"] 
    df["x5"] = (df["Close"] - df["Low"]) / df["Close"] 

    return df.drop(["Open", "High", "Low", "Date", "Close"], axis=1)

    

In [102]:
#timeframes = ["1m", "5m", "15m", "30m", "1H", "4H", "1D"]
used_timeframes = ["15m", "1H"]
    
query = Query(episode_length=256, trading_timeframe="15m", trading_column="Close")
# query.add_query(
#     timeframe="4H",
#     window_size=4,#     data_processor=article_processor
# )
query.add_query(
    timeframe="1H",
    window_size=2,
    data_processor=select_date
)
query.add_query(
    timeframe="15m",window_size=2,
    data_processor=select_date
)

timeframes = CurrencyPair(ticker="EURUSD", timeframes=used_timeframes).timeframes


format_datasets(query, timeframes).head()

,Date,Trading_Price,1H_Date_0,1H_Close_0,1H_Date_1,1H_Close_1,15m_Date_0,15m_Close_0,15m_Date_1,15m_Close_1
0,2023-09-20 02:00:00+00:00,1.06898,2023-09-20 01:00:00+00:00,1.06864,2023-09-20 00:00:00+00:00,1.06850,2023-09-20 02:00:00+00:00,1.06898,2023-09-20 01:45:00+00:00,1.06864
1,2023-09-20 02:15:00+00:00,1.06862,2023-09-20 01:00:00+00:00,1.06864,2023-09-20 00:00:00+00:00,1.06850,2023-09-20 02:15:00+00:00,1.06862,2023-09-20 02:00:00+00:00,1.06898
2,2023-09-20 02:30:00+00:00,1.06856,2023-09-20 01:00:00+00:00,1.06864,2023-09-20 00:00:00+00:00,1.06850,2023-09-20 02:30:00+00:00,1.06856,2023-09-20 02:15:00+00:00,1.06862
3,2023-09-20 02:45:00+00:00,1.06832,2023-09-20 01:00:00+00:00,1.06864,2023-09-20 00:00:00+00:00,1.06850,2023-09-20 02:45:00+00:00,1.06832,2023-09-20 02:30:00+00:00,1.06856
4,2023-09-20 03:00:00+00:00,1.06824,2023-09-20 02:00:00+00:00,1.06832,2023-09-20 01:00:00+00:00,1.06864,2023-09-20 03:00:00+00:00,1.06824,2023-09-20 02:45:00+00:00,1.06832


In [18]:
timeframes["15m"]["Close"].shift(-1)

0        1.06835
1        1.06837
2        1.06850
3        1.06851
4        1.06852
          ...   
25074    1.11581
25075    1.11577
25076    1.11629
25077    1.11641
25078        NaN
Name: Close, Length: 25079, dtype: float64

In [11]:
ohlc = pd.DataFrame(columns=["Date", "Open", "High", "Low", "Close"])
ohlc["Date"] = pd.date_range(start="2023-09-20 02:00:00+00:00", end="2023-10-20 03:00:00+00:00", freq="15min")
ohlc["Open"] = list(range(0, 2885))
ohlc["High"] = list(range(0, 2885))
ohlc["Low"] = list(range(0, 2885))
ohlc["Close"] = list(range(0, 2885))
ohlc.tail()

,Date,Open,High,Low,Close
2880,2023-10-20 02:00:00+00:00,2880,2880,2880,2880
2881,2023-10-20 02:15:00+00:00,2881,2881,2881,2881
2882,2023-10-20 02:30:00+00:00,2882,2882,2882,2882
2883,2023-10-20 02:45:00+00:00,2883,2883,2883,2883
2884,2023-10-20 03:00:00+00:00,2884,2884,2884,2884


In [12]:
from forexgym.utils import query

In [13]:
def select_close(df: pd.DataFrame, *args, **kwargs) -> pd.DataFrame:
    
    return pd.DataFrame(df["Close"])

def article_processor(df: pd.DataFrame, *args, **kwargs) -> pd.DataFrame:
    df["x1"] = ((df["Close"] - df["Close"].shift(1) ) / df["Close"]).shift(1) 
    df["x2"] = ((df["High"] - df["High"].shift(1) ) / df["High"]).shift(1) 
    df["x3"] = ((df["Low"] - df["Low"].shift(1) ) / df["Low"]).shift(1) 
    df["x4"] = (df["High"] - df["Close"]) / df["Close"] 
    df["x5"] = (df["Close"] - df["Low"]) / df["Close"] 
    
    return df.drop(["Open", "High", "Low", "Close", "Date"], axis=1)

In [ ]:
#timeframes = ["1m", "5m", "15m", "30m", "1H", "4H", "1D"]
timeframes = ["15m", "1H"]

query = Query(episode_length=256, trading_timeframe="15m", trading_column="Close")

query.add_query(
    timeframe="1H",
    window_size=1,
    data_processor=article_processor
)

In [15]:
query.observation_size

5

In [3]:
ohlc = pd.DataFrame(columns=["Date", "Open", "High", "Low", "Close"])
start = datetime.datetime(2023, 9, 20, 2)
end = start + datetime.timedelta(hours=10)
ohlc["Date"] = pd.date_range(start=start, end=end, freq="15min")

In [4]:
ohlc.head(8)

,Date,Open,High,Low,Close
0,2023-09-20 02:00:00,NaN,NaN,NaN,NaN
1,2023-09-20 02:15:00,NaN,NaN,NaN,NaN
2,2023-09-20 02:30:00,NaN,NaN,NaN,NaN
3,2023-09-20 02:45:00,NaN,NaN,NaN,NaN
4,2023-09-20 03:00:00,NaN,NaN,NaN,NaN
5,2023-09-20 03:15:00,NaN,NaN,NaN,NaN
6,2023-09-20 03:30:00,NaN,NaN,NaN,NaN
7,2023-09-20 03:45:00,NaN,NaN,NaN,NaN


In [20]:
article_processor(ohlc)

,Volume,x1,x2,x3,x4,x5
0,0,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,0.0,0.0
2,2,1.000000,1.000000,1.000000,0.0,0.0
3,3,0.500000,0.500000,0.500000,0.0,0.0
4,4,0.333333,0.333333,0.333333,0.0,0.0
...,...,...,...,...,...,...
2880,2880,0.000347,0.000347,0.000347,0.0,0.0
2881,2881,0.000347,0.000347,0.000347,0.0,0.0
2882,2882,0.000347,0.000347,0.000347,0.0,0.0
2883,2883,0.000347,0.000347,0.000347,0.0,0.0
